In [2]:
!pip install python-dotenv

In [10]:
from os import getenv
import asyncio

import pandas as pd
import requests
from dotenv import load_dotenv
import time

In [11]:
load_dotenv()
TOKEN = getenv('tokenEmpresas')

In [12]:
df = pd.read_csv("/content/RelatorioInfoB2B_MapaParqueVisaoCliente_20240425094745.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
df = df[df['TIPO_DOCUMENTO'] == "CNPJ"]

In [13]:
df['NR_CNPJ'] = df['NR_CNPJ'].astype(str).str.zfill(14)

In [14]:
cnpj_dados_nao_encontrados = []

In [15]:
async def fetch_data_empresas(i, cnpj):
  URL = f"https://empresaqui.com.br/api/{TOKEN}/{cnpj}"

  response = requests.get(URL)
  if response.status_code == 200:
    try:
      data = response.json()
      df.at[i, 'uf'] = data.get('log_uf')
      df.at[i, 'municipio'] = data.get('log_municipio')
      df.at[i, 'cep'] = data.get('log_cep')
      df.at[i, 'tipo endereco'] = data.get('log_tipo')
      df.at[i, 'bairro'] = data.get('log_bairro')
      df.at[i, 'ddd'] = data.get('ddd')

      df.at[i, 'cnae'] = data.get('cnae_principal')
      df.at[i, 'matriz'] = data.get('matriz')
      df.at[i, 'natureza_juridica'] = data.get('natureza_juridica')
      df.at[i, 'data_abertura'] = data.get('data_abertura')
      df.at[i, 'data_sit_cad'] = data.get('data_sit_cad')

      df.at[i, 'porte'] = data.get('porte')
      df.at[i, 'capital_social'] = data.get('capital_social')
      df.at[i, 'regime_tributario'] = data.get('regime_tributario')
      df.at[i, 'faturamento'] = data.get('faturamento')
      df.at[i, 'quadro_funcionarios'] = data.get('quadro_funcionarios')

      print("sucesso")

    except:
      cnpj_dados_nao_encontrados.append(cnpj)
      print("deu erro")

  else:
    cnpj_dados_nao_encontrados.append(cnpj)
    print("deu ruim")

In [22]:
async def main():
    tasks = []
    for i, row in df.iterrows():
        cnpj = row['NR_CNPJ']
        if cnpj in cnpj_dados_nao_encontrados:
          tasks.append(asyncio.create_task(fetch_data_empresas(i, cnpj)))

In [ ]:
start_time = time.time()
await main()
end_time = time.time()

In [27]:
len(cnpj_dados_nao_encontrados)

6242

In [28]:
df.to_excel("atualizada2.xlsx")